In [1]:
import pandas as pd
import numpy as np

In [13]:
def get_params(data, start, emotional):
    # номер диaлога и тональность
    nums=[]
    num = start
    for i, row in data.iterrows():
        if row["№ сообщения"]==1:
            num+=1
        nums.append(num)
    data['№ диалога']=nums
    data['emotional']=emotional
    return data

In [36]:
def create_texts(data):
    # тексты пользователя для датасета
    texts=[]
    string = ''
    for i, row in data.iterrows():
        if row["№ сообщения"]==1:
            if string != '':
                texts.append(string)
                string = str(row["Текст"])+' '
            else:
                string += str(row["Текст"])+' '
        else:
            if row["Направление"]=='in':
                string += str(row["Текст"])+' '
    texts.append(string)
    return texts

In [11]:
df_pos = pd.read_excel('Газпром_датасет.xlsx', sheet_name='Целевая', engine='openpyxl')

In [16]:
df_neutral = pd.read_excel('Газпром_датасет.xlsx', sheet_name='Нецелевая', engine='openpyxl')

In [18]:
df_neg = pd.read_excel('Газпром_датасет.xlsx', sheet_name='Негативная', engine='openpyxl')

In [37]:
df_train = pd.concat([get_params(df_pos, 0, "positive"), get_params(df_neutral, 39, "neutral"),
                     get_params(df_neg, 77, "negative")], ignore_index=True)

In [38]:
df_train

,№ сообщения,Текст,Направление,№ диалога,emotional
0,1,Где информация о вкладе 13%,in,1,positive
1,2,Добрый день! Я – виртуальный помощник 😉 Услови...,out,1,positive
2,3,Да,in,1,positive
3,4,Вас интересуют условия действующих продуктов и...,out,1,positive
4,5,Оформить новый,in,1,positive
...,...,...,...,...,...
1737,8,"Выберите, что вас интересует:",out,119,negative
1738,9,"Соедини с оператором, на твои дебильные вопрос...",in,119,negative
1739,10,Ожидание специалиста может занять длительное в...,out,119,negative
1740,11,Оператор,in,119,negative


In [43]:
temp = df_train[['№ диалога', 'emotional']].drop_duplicates()
temp['texts']=create_texts(df_train)
temp['отказ от продуктов']=0
temp['жалобы']=0
temp['просроченная задолженность']=0
temp['мошенничество']=0
temp['утеря/кража карты']=0
temp

,№ диалога,emotional,texts,отказ от продуктов,жалобы,просроченная задолженность,мошенничество,утеря/кража карты
0,1,positive,Где информация о вкладе 13% Да Оформить новый ...,0,0,0,0,0
13,2,positive,Свяжите с оператором Оформление нового продукт...,0,0,0,0,0
27,3,positive,Свяжите с оператором Оформление нового продукт...,0,0,0,0,0
41,4,positive,Здравствуйте! Можно ли открыть долгосрочный вк...,0,0,0,0,0
55,5,positive,Добрый день. Я сняла деньги с накопительного с...,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1701,115,negative,Вы издеваетесь? Просто самый худший банк!!!!! ...,0,0,0,0,0
1707,116,negative,Я задала конкретный ясный вопрос. Зачем пересп...,0,0,0,0,0
1722,117,negative,"Чо за бред, изначально была речь про пожизненн...",0,0,0,0,0
1726,118,negative,Желаю воспользоваться кредиткой и бегать с хер...,0,0,0,0,0


В негативе есть отказ от продуктов и жалобы. Добавим для разметки: 'просроченная задолженность', 'мошенничество', 'утеря/кража карты'

In [50]:
texts = ['Имеет ли право кредитор звонить мне по задолженности? Получается, я не могу отказаться от общения с банком? О чем может спрашивать меня сотрудник банка, если у меня есть просроченная задолженность по кредиту? С каких номеров может звонить банк по вопросу задолженности?', 
        'Что будет если просрочить кредит. У меня и правда есть проблемы с кредитом — я забыл или не смог вовремя его оплатить. Я пропустил всего один платёж, разве это так уж важно? Но мне сейчас сложно погасить долг. У меня ухудшилось финансовое положение, и я хочу расторгнуть кредитный договор.',
        "Какими способами банк может взаимодействовать со мной? Что за реструктуризация и кредитные каникулы и ипотечные каникулы? А если банк уже передал мой долг коллектору? Я хочу объявить себя банкротом, чтобы банк списал мне все долги.",
        'Куда обращаться в случае мошенничества с банковской картой? Как отличить мошенников от сотрудников банка? Откуда у мошенников мой номер?',
        ' Кто может помочь вернуть деньги от мошенников? Что делать если взял кредит и перевел деньги мошенникам? Можно ли отменить операцию по банковской карте?',
        'Можно ли найти мошенника по номеру банковской карты? Что делать если перевели деньги и просят вернуть? Можно ли вернуть деньги снятые с карты мошенниками? ',
        'Не могу найти карту. Что делать? Что если карта утеряна или ее украли? Что делать, если украли деньги с карты? Как не допустить потерю или кражу карты? Что делать, если воры украли карту и потратили деньги?',
        'Как заблокировать карту? Как перевыпустить карту? Как восстановить банковскую карту? Сколько восстанавливают карту? Сколько стоит восстановить карту? Что делать, если карта нашлась после блокировки?',
        'Как снять деньги с утерянной карты? Контроль движения средств по карте. Что делать, если карта нашлась после блокировки или перевыпуска? Что могут сделать мошенники с найденной картой.']

d = {'№ диалога': [120, 121, 122, 123, 124, 125, 126, 127, 128], 
     'emotional': ['neutral','neutral','neutral', 'neutral','neutral','neutral', 'neutral','neutral','neutral'], 
     'texts': texts, 'отказ от продуктов':[0,0,0,0,0,0,0,0,0], 'жалобы':[0,0,0,0,0,0,0,0,0],
     'просроченная задолженность':[1,1,1,0,0,0,0,0,0], 'мошенничество':[0,0,0,1,1,1,0,0,0], 
     'утеря/кража карты':[0,0,0,0,0,0,1,1,1]}
temp1 = pd.DataFrame(data=d)
temp1

,№ диалога,emotional,texts,отказ от продуктов,жалобы,просроченная задолженность,мошенничество,утеря/кража карты
0,120,neutral,Имеет ли право кредитор звонить мне по задолже...,0,0,1,0,0
1,121,neutral,Что будет если просрочить кредит. У меня и пра...,0,0,1,0,0
2,122,neutral,Какими способами банк может взаимодействовать ...,0,0,1,0,0
3,123,neutral,Куда обращаться в случае мошенничества с банко...,0,0,0,1,0
4,124,neutral,Кто может помочь вернуть деньги от мошенников...,0,0,0,1,0
5,125,neutral,Можно ли найти мошенника по номеру банковской ...,0,0,0,1,0
6,126,neutral,Не могу найти карту. Что делать? Что если карт...,0,0,0,0,1
7,127,neutral,Как заблокировать карту? Как перевыпустить кар...,0,0,0,0,1
8,128,neutral,Как снять деньги с утерянной карты? Контроль д...,0,0,0,0,1


In [52]:
df = pd.concat([temp, temp1], ignore_index=True)
df

,№ диалога,emotional,texts,отказ от продуктов,жалобы,просроченная задолженность,мошенничество,утеря/кража карты
0,1,positive,Где информация о вкладе 13% Да Оформить новый ...,0,0,0,0,0
1,2,positive,Свяжите с оператором Оформление нового продукт...,0,0,0,0,0
2,3,positive,Свяжите с оператором Оформление нового продукт...,0,0,0,0,0
3,4,positive,Здравствуйте! Можно ли открыть долгосрочный вк...,0,0,0,0,0
4,5,positive,Добрый день. Я сняла деньги с накопительного с...,0,0,0,0,0
...,...,...,...,...,...,...,...,...
123,124,neutral,Кто может помочь вернуть деньги от мошенников...,0,0,0,1,0
124,125,neutral,Можно ли найти мошенника по номеру банковской ...,0,0,0,1,0
125,126,neutral,Не могу найти карту. Что делать? Что если карт...,0,0,0,0,1
126,127,neutral,Как заблокировать карту? Как перевыпустить кар...,0,0,0,0,1


In [53]:
#после сохранения вручную разметить отказ и жалобы
df.to_csv("train.csv", index=False)